In [6]:
d='done'
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import time
import os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(d)

done


In [5]:
date_api_max = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
date_hier = datetime.now() - timedelta(days=1)
date_api_min = date_hier.strftime("%Y-%m-%dT%H:%M:%SZ")
#------------------------------------------------------------------------------------------------------------------------------#
today = datetime.now()
aujourdhui = today.strftime("%d-%m-%Y")
today = today.strftime("%d%b%Y").upper()

file_path = f'C:\\*****\\****\\*******\\***\\*********\\***********\\{today}'

# Crée le dossier d'aujourd'hui
if not os.path.exists(file_path):
    os.makedirs(file_path)

key_words = ['Data','Data analyst','data engineer','data scientist',"Consultant décisionnel",
             'Business Intelligence',"Consultant informatique","Testeur",
             'consultant bi','java','django','power bi','python','statistique','big data',
             'développeur','statisticien','sap bo',"systeme d'information"]

   
appellation_poste = [ 'Data analyst',
                      'Analyste décisionnel - Business Intelligence',
                      'Développeur(se) décisionnel - Business Intelligence',
                      "Chargé / Chargée d'études statistiques",
                      'Développeur / Développeuse Big Data',
                      'Consultant / Consultante décisionnel - Business Intelligence',
                      'Statisticien / Statisticienne',
                      'Data miner','Data scientist',]
#------------------------------------------------------------------------------------------------------------------------------#
# Vos informations d'identification
client_id = '********************************************'
client_secret = '*************************'

# Les données pour la requête
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'api_offresdemploiv2 o2dsoffre'
}

# L'en-tête de la requête
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# L'URL pour obtenir le token, avec le paramètre 'realm' inclus dans l'URL
url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=%2Fpartenaire"
ids = [] 

for i in range(1):
	time.sleep(0)
	# Envoyer la requête
	response = requests.post(url, data=data, headers=headers)
	#print(response)
	token_data = response.json()
	#print(token_data)
	access_token = token_data['access_token']
	#print(access_token)

	# Votre token d'accès
	token = access_token
	# Fonction pour récupérer les offres
	def recuperer_offres(token, key_word, range_debut, range_fin):
		url = 'https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search'
		headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
		params = {'motsCles': key_word, 
                  'range': f'{range_debut}-{range_fin}',
                  'minCreationDate': date_api_min,
                  'maxCreationDate': date_api_max}
		response = requests.get(url, headers=headers, params=params)

		if response.status_code in [200, 206]:
			return response.json()['resultats']
		else:
			print("Erreur lors de la requête :", response.status_code, response.text)
			return []
         
	offres_totales = []
	for key_word in key_words:
		range_debut, range_fin = 0, 149
		max_range = 3000  # Limite maximale autorisée pour le début de la plage

		while range_debut <= max_range:
			ids = list(set(ids))
			offres = recuperer_offres(token, key_word, range_debut, range_fin)
			for offre in offres:
				if offre['id'] not in ids:
					offres_totales.append(offre)
					ids.append(offre['id'])
			#print(ids)        
                    
            
			# Si moins de 150 offres sont retournées, c'est la dernière page
			if len(offres) < 150:
				break

			range_debut += 150
			range_fin += 150
			range_fin = min(range_fin, 3149)  

		print(f'Fin de la récupération des offres pour le mot-clé : {key_word}')
        

	print(f'nb_offres : {len(offres_totales)}')
	if len(offres_totales) != 0:
		donnees = [{
        'id': offre['id'],
        'poste': offre['intitule'],
        'description': offre['description'].lower(),
        'date_creation': datetime.strptime(offre['dateCreation'], "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%d-%m-%Y") 
            if 'dateCreation' in offre else None,
        'date_actualisation': datetime.strptime(offre['dateActualisation'], "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%d-%m-%Y") 
            if 'dateActualisation' in offre else None,
        'lieu': offre['lieuTravail']['libelle'] 
            if 'lieuTravail' in offre and 'libelle' in offre['lieuTravail'] else None,
        'salaire': offre['salaire']['libelle'] 
            if 'salaire' in offre and 'libelle' in offre['salaire'] else None,
        'appellation': offre['appellationlibelle'],
        'entreprise': offre['entreprise']['nom'] 
            if 'entreprise' in offre and 'nom' in offre['entreprise'] else None,
        'contrat': offre['typeContrat'],
        'experience': offre['experienceLibelle'],
        #'nb_postes': offre['nombrePostes'],
        'urlOrigine': offre['origineOffre']['urlOrigine'] 
            if 'origineOffre' in offre and 'urlOrigine' in offre['origineOffre'] else None,
        'partenaire': offre['origineOffre']['partenaires'][0]['nom'] 
            if 'origineOffre' in offre and 'partenaires' in offre['origineOffre'] 
            and len(offre['origineOffre']['partenaires']) > 0 else None,
         'url_partenaire': offre['origineOffre']['partenaires'][0]['url']
            if 'origineOffre' in offre and 'partenaires' in offre['origineOffre'] 
            and len(offre['origineOffre']['partenaires']) > 0 else None

		} for offre in offres_totales]
    
		# Conversion en DataFrame
		df_01 = pd.DataFrame()
		df_01 = pd.DataFrame(donnees)
		df_01 = df_01.drop_duplicates(subset='id', keep='first')
		df_01 = df_01[df_01['appellation'].isin(appellation_poste)]
    
		email_regex = r'([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})'
		df_01['email'] = df_01['description'].str.extract(email_regex)
    
		nb_offres = len(df_01)
		print(nb_offres)
		if nb_offres > 0:       
			#export
			current_time = datetime.now().strftime("%Hh%M")
			df_01.to_excel(f'{file_path}\\{current_time} - {today}.xlsx')
            
	print(f'Exécution N°{i} terminé')
#------------------------------------------------------------------------------------------------------------------------------#
print(d)
df_01.head()

Fin de la récupération des offres pour le mot-clé : Data
Fin de la récupération des offres pour le mot-clé : Data analyst
Fin de la récupération des offres pour le mot-clé : data engineer
Fin de la récupération des offres pour le mot-clé : data scientist
Fin de la récupération des offres pour le mot-clé : Consultant décisionnel
Fin de la récupération des offres pour le mot-clé : Business Intelligence
Fin de la récupération des offres pour le mot-clé : Consultant informatique
Fin de la récupération des offres pour le mot-clé : Testeur
Fin de la récupération des offres pour le mot-clé : consultant bi
Fin de la récupération des offres pour le mot-clé : java
Fin de la récupération des offres pour le mot-clé : django
Fin de la récupération des offres pour le mot-clé : power bi
Fin de la récupération des offres pour le mot-clé : python
Fin de la récupération des offres pour le mot-clé : statistique
Fin de la récupération des offres pour le mot-clé : big data
Fin de la récupération des offres

,id,poste,description,date_creation,date_actualisation,lieu,salaire,appellation,entreprise,contrat,experience,urlOrigine,partenaire,url_partenaire,email
5,172JDQK,Ingénieur Big Data (H/F),dans le cadre de projets big data menés par da...,04-04-2024,04-04-2024,06 - VALBONNE,Annuel de 38000.0 Euros à 42000.0 Euros sur 12...,Développeur / Développeuse Big Data,DAVIDSON PACA,CDI,2 An(s),https://candidat.pole-emploi.fr/offres/recherc...,None,None,NaN
6,172HJGN,DATA ANALYST (H/F),nous recherchons notre data analyst h/f pour i...,04-04-2024,04-04-2024,79 - NIORT,Annuel de 35000 Euros à 42000 Euros sur 13 mois,Data analyst,INETUM,CDI,3 An(s),https://candidat.pole-emploi.fr/offres/recherc...,None,None,NaN
12,4587580,Data analyst (H/F),responsabilités : \n\nles missions d'un amilto...,05-04-2024,05-04-2024,59 - LILLE,"Annuel de 40000,00 Euros à 45000,00 Euros",Data analyst,Amiltone,CDI,Débutant accepté,https://candidat.pole-emploi.fr/offres/recherc...,BEETWEEN,https://emploi.beetween.com/apply/job/dym054s2...,NaN
14,4584626,"Data Analyst H.F F/H - Système, réseaux, donné...",descriptif du poste:\n\nvos challenges&nbsp;au...,05-04-2024,05-04-2024,13 - MARSEILLE 08,None,Data analyst,MUTUELLES DU SOLEIL LIVRE II,CDI,Expérience exigée de 2 An(s),https://candidat.pole-emploi.fr/offres/recherc...,APEC,https://www.apec.fr/candidat/recherche-emploi....,NaN
16,4583009,Data Analyst / Consultant (H/F),la dimension de la gestion de projet vous tent...,05-04-2024,05-04-2024,44 - NANTES,None,Data analyst,Societe Generale,CDD,Débutant accepté,https://candidat.pole-emploi.fr/offres/recherc...,BROADBEAN,https://www.aplitrak.com/?adid=UkVDU0dDZW50cmF...,NaN
